# Data Cleaning & Preprocessing

In [ ]:
import numpy as np
import pandas as pd
import statistics as stats
import re
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')
fifa21_df_org=pd.read_csv("fifa21_train.csv")

In [ ]:
def standardize_headers(df):
    df.columns = [col.lower().replace(' ','_') for col in df.columns]
    return df

In [ ]:
def convert_ftin_to_cm(df,col_name):
    df[col_name]=df[col_name].apply(lambda x : int(x.split("'")[0])*30.48+int(x.split("'")[1].replace("\"",""))*2.54)
    df[col_name]=df[col_name].apply(lambda x : round(x,0))
    return df

In [ ]:
def convert_lbs_to_kg(df,col_name):
    df[col_name]=df[col_name].apply(lambda x : round(int(x.replace("lbs",""))*0.45359237,0))
    return df

In [ ]:
def replace_datecol_with_yearcol(df,col_name):
    df[col_name]=pd.to_datetime(df[col_name],errors='coerce')
    df[col_name+'_'+'year']=df[col_name].apply(lambda x : x.year)

    # Drop position of the joined column
    df.drop([col_name], axis=1,inplace=True)

    # Check how many are NaN
    df[col_name+'_'+'year'].isna().sum()

    # Keep only the non NaN for joined_year
    df = df[df[col_name+'_'+'year'].notna()]

    # Change joined_year to int
    df[col_name+'_'+'year']=df[col_name+'_'+'year'].apply(lambda x : int(x))
    
    return df

In [ ]:
def convert_currency_col_to_int(df,col_name):
    df[col_name]=df[col_name].apply(lambda x : int(x.replace("€","").replace("K","000").replace("M","000000").replace(".","")))
    return df

In [ ]:
def split_year_range_to_cols(df,col_name):
    df[col_name+'_start'] = ""
    df[col_name+'_end'] = ""
    df[col_name+'_start']=df[col_name].apply(lambda x : x.split("~")[0].strip())
    df[col_name+'_end']=df[col_name].apply(lambda x : np.nan if x.find('~') == -1 else x.split("~")[1].strip())
    return df

In [ ]:
def extract_year_from_str_col(df,col_name):
    df[col_name]=df[col_name].apply(lambda x : re.findall("\d{4}",x)[0] if len(re.findall("\d{4}",x))>0 else np.nan)
    return df

In [ ]:
def sumup_operands(df,col_name):
    df[col_name]=df[col_name].apply(lambda x : int(x.split("+")[0])+int(x.split("+")[1]))
    return df

In [ ]:
def identify_cols_nan(df):
    df=pd.DataFrame(df.isna().sum(),columns=['count'])
    return df[df['count']>0]

In [ ]:
def preprocess(df,filename):
    # 1. Standardize header names.
    df=standardize_headers(df)
    
    # 2. Drop position of the following columns
    df.drop(['id','name','position','team_&_contract','loan_date_end'], axis=1,inplace=True)
    
    # 3. Converting Height to cm
    df=convert_ftin_to_cm(df,'height')
    
    # 4. Convert weight in lbs to kgs
    df=convert_lbs_to_kg(df,'weight')
    
    # 5. Convert joined col to Datetime
    df=replace_datecol_with_yearcol(df,'joined')
    
    # 6. Convert value, wage and release_clause to float
    df=convert_currency_col_to_int(df,'value')
    df=convert_currency_col_to_int(df,'wage')
    df=convert_currency_col_to_int(df,'release_clause')
    
    # 7. Add contract start and end date
    df=split_year_range_to_cols(df,'contract')
    
    # 8. Add column is_on_loan
    df['is_on_loan']=df['contract_start'].apply(lambda x : 0 if x.find('On Loan')==-1 else 1)
    
    # 9. Eliminate the ★ and convert to int
    df['w/f']=df['w/f'].apply(lambda x : int(x.replace("★","")))
    df['sm']=df['sm'].apply(lambda x : int(x.replace("★","")))
    df['ir']=df['ir'].apply(lambda x : int(x.replace("★","")))
    
    # 10. Extract contract start date from contract_start column containing other strings
    df=extract_year_from_str_col(df,'contract_start')
    
    # 11. Convert columns contract_start and contract_end to int
    df['contract_start']=pd.to_numeric(df['contract_start'],errors='coerce')
    df['contract_end']=pd.to_numeric(df['contract_end'],errors='coerce')
    # Drop column contract
    df.drop(['contract'], axis=1,inplace=True)
    
    # 12. Convert other string columns containing only ints
    df.select_dtypes(include=object)
    df['hits']=pd.to_numeric(df['hits'],errors='coerce')
    
    # 13. Convert skill columns to numeric after adding the bonus to a single integer value
    skill_cols=['ls','st','rs','lw','lf','cf','rf','rw','lam','cam','ram','lm','lcm','cm','rcm','rm','lwb','ldm','cdm','rdm','rwb','lb','lcb','cb','rcb','rb','gk']
    for col in skill_cols:
        df=sumup_operands(df,col)
        
    # 14. Check for NaNs
    nan_rep_df=identify_cols_nan(df)
    
    # 15. Handling of NaNs for below columns:
    
    # We choose median value for Nan values in composure
    df['composure'].value_counts()
    df['composure'] = df['composure'].fillna(stats.median(df['composure']))

    # We choose mode value for Nan values in a/w
    df['a/w'].value_counts()
    df['a/w'] = df['a/w'].fillna(stats.mode(df['a/w']))
    
    # We choose mode value for Nan values in d/w
    df['d/w'].value_counts()
    df['d/w'] = df['d/w'].fillna(stats.mode(df['d/w']))
    
    # We choose mode value for Nan values in hits
    df['hits'].value_counts()
    df['hits'] = df['hits'].fillna(stats.mode(df['hits']))
    
    # Remove Nans
    df = df[df['contract_start'].notna()]
    
    # Export the cleaned dataset:
    df.to_csv(filename+".csv",index=False)
    
    return df
    

In [ ]:
fifa21_df=preprocess(fifa21_df_org,'fifa21_df_cleaned')

# EDA

In [ ]:
# 2. Describe DataFrame.
fifa21_df.describe().T

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

In [ ]:
# Barplots
fifa21_df['foot'].value_counts().plot(kind='bar')

# INFERENCE: Right Foot Players are dominant

In [ ]:
fifa21_df['bp'].value_counts().plot(kind='bar')
# INFERENCE: Center Back is the most dominant position

In [ ]:
fifa21_df['a/w'].value_counts().plot(kind='bar')

# INFERENCE: Medium Attacking Work Rate is the most dominant value.

In [ ]:
fifa21_df['d/w'].value_counts().plot(kind='bar')

# INFERENCE: Medium Defensive Work Rate is the most dominant value.

In [ ]:
fifa21_df['joined_year'].value_counts().plot(kind='bar')

# INFERENCE: Most players joined back in 2019.

In [ ]:
sns.kdeplot(fifa21_df['age'])

# INFERENCE: Most players are in between 20's to 22s

In [ ]:
sns.kdeplot(fifa21_df['weight'])
# INFERENCE: Most players have an average weight of 75kgs

In [ ]:
sns.regplot(x=fifa21_df["age"], y=fifa21_df["ova"])
# INFERENCE: A positive correlation between age and overall rating, although dispersed

In [ ]:
sns.regplot(x=fifa21_df["skill"], y=fifa21_df["ova"])
# INFERENCE: 2 distinct clusters of scattered points, although showing a slight positive correlation of skill and overall rating.

In [ ]:
sns.regplot(x=fifa21_df["total_stats"], y=fifa21_df["ova"])
# INFERENCE: 2 distinct clusters of scattered points, although showing a slight positive correlation of total_stats and overall rating.

In [ ]:
sns.regplot(x=fifa21_df["base_stats"], y=fifa21_df["ova"])
# INFERENCE: A strong positive correlation between base_stats and overall rating

In [ ]:
sns.regplot(x=fifa21_df["composure"], y=fifa21_df["ova"])
# INFERENCE: A strong positive correlation between composure and overall rating, with bit of dispersion

In [ ]:
sns.regplot(x=fifa21_df["reactions"], y=fifa21_df["ova"])
# INFERENCE: A strong positive correlation between reactions and overall rating

In [ ]:
sns.regplot(x=fifa21_df["growth"], y=fifa21_df["ova"])
# INFERENCE: A strong negative correlation between growth and overall rating, with ordered dispersion at distinct growth values.

# Feature Selection & Model Building

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn import linear_model
import pickle
isTrain=True

In [ ]:
# Loading the cleaned/transformed fifa dataset from the previous stage
fifa21_df=pd.read_csv("fifa21_df_cleaned.csv")
# Dropping the unnecessary columns
fifa21_df = fifa21_df.drop(['contract_start','contract_end','nationality','club'], axis=1)

In [ ]:
# Function definition for preparation/preprocessing of model input

def preprocess_model_input(df,output_col=None,is_train_test_split=False):
    # 1. Split the numerical, categorical and the output column:
    X_num=df.select_dtypes(include=np.number)
    X_cat=df.select_dtypes(include=object)
    if output_col is not None:
        y = X_num[output_col]
        X_num = X_num.drop([output_col], axis=1)
    
    # 2. Normalizing data:
    transformer = MinMaxScaler().fit(X_num)
    X_normalized = transformer.transform(X_num)
    X_normalized = pd.DataFrame(X_normalized,columns=X_num.columns)
    
    # 3. Encoding categorical columns:
    if len(X_cat)>0:
        encoder = OneHotEncoder().fit(X_cat)
        encoded = encoder.transform(X_cat).toarray()
        cols = encoder.get_feature_names_out(input_features=X_cat.columns)
        onehot_encoded = pd.DataFrame(encoded, columns=cols)

    # 4. Concatenating normalized numeric columns and encoded categorical columns

    if len(X_cat)>0:
        X = pd.concat([X_normalized, onehot_encoded], axis=1)
    
    else:
        X = pd.concat([X_normalized], axis=1)
        
    # 5. Creating a Train-Test Split
    if is_train_test_split:

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21)

        return X_train, X_test, y_train, y_test
    else:
        if output_col is not None:
            return X,y
        else:
            return X

# Function for evaluating the model and displaying the metrics of the model accuracy

def produce_metrics_lm(lm,X_test,y_test,isPrint=False):
    predictions_test = lm.predict(X_test)
    r2=round(r2_score(y_test, predictions_test),2)
    mas=round(mean_absolute_error(y_test, predictions_test),2)
    mse=round(mean_squared_error(y_test,predictions_test),2)
    rmse=round(np.sqrt(mean_squared_error(y_test,predictions_test)),2)
    if isPrint:
        print("r2_score: ",r2)
        print("mean absolute error :",mas)
        print("mean square error :",mse)
        print("root mean square error :",rmse)
    else:
        return r2,mas,mse,rmse,predictions_test

In [ ]:
######### FEATURE SELECTION ##################
# (involves selection methods: brute force (manual selection trials) + correlation matrix (along with identifying 
# Multicollinearity)) - total of 39 features + 1 output variable

# A selection of 46 features was giving a raise to the accuracy by only +0.02, therefore we decided to stick with only 39 
# features as below to limit the size of model.

selected_features=[
                    'age',
                    'height',
                    'weight',
                    'value',
                    'wage',
                    'release_clause',
                    'attacking',
                    'crossing',
                    'finishing',
                    'heading_accuracy',
                    'short_passing',
                    'volleys',
                    'skill',
                    'dribbling',
                    'curve',
                    'fk_accuracy',
                    'long_passing',
                    'ball_control',
                    'movement',
                    'acceleration',
                    'sprint_speed',
                    'agility',
                    'reactions',
                    'balance',
                    'power',
                    'shot_power',
                    'jumping',
                    'stamina',
                    'strength',
                    'long_shots',
                    'mentality',
                    'aggression',
                    'interceptions',
                    'positioning',
                    'vision',
                    'penalties',
                    'composure',
                    'marking',
                    'goalkeeping',
                    'ova'
                    ]

fifa21_df_selected=fifa21_df[selected_features]

X_train, X_test, y_train, y_test = preprocess_model_input(fifa21_df_selected,'ova',True)

In [ ]:
# Perform the training of the dataset on Linear Regression Model and save the model
isTrain=True
# Creating and fitting a Linear Regression Model
lm = linear_model.LinearRegression()
lm.fit(X_train,y_train)
if isTrain==True:
    filename = 'fifa21_lm.sav'
    pickle.dump(lm, open(filename, 'wb'))
else:
    lm = pickle.load(open(filename, 'rb'))

In [ ]:
# Perform the model evaluation for Training and Test Dataset
print("Training Metrics:")
produce_metrics_lm(lm,X_train,y_train,True)
print()
print("Test Metrics:")
produce_metrics_lm(lm,X_test,y_test,True)

# Validation Test

In [ ]:
# Load the validation dataset
isTrain=False
fifa21_df_validation=pd.read_csv("fifa21_validate.csv")

In [ ]:
# Preprocess the validation dataset for the cleaning and standardization, and drop the unnecessary columns
fifa21_df_validation_cleaned=preprocess(fifa21_df_validation,'fifa21_df_valid_cleaned')
fifa21_df_validation_cleaned = fifa21_df_validation_cleaned.drop(['contract_start','contract_end','nationality','club'], axis=1)

In [ ]:
# Extract the necessary features needed for performing the prediction and later preprocess the DF for model preparation
fifa21_df_validation_cleaned_selected=fifa21_df_validation_cleaned[selected_features]
X_valid,y_valid=preprocess_model_input(fifa21_df_validation_cleaned_selected,'ova',False)

In [ ]:
# Loading the Linear Regression Model
if isTrain==True:
    filename = 'fifa21_lm.sav'
    pickle.dump(lm, open(filename, 'wb'))
else:
    lm = pickle.load(open(filename, 'rb'))

In [ ]:
# Perform the model evaluation for Validation Dataset
print("Validation Metrics:")
produce_metrics_lm(lm,X_valid,y_valid,True)